In [1]:
import os
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv('Charades_vu17_train.csv')

In [3]:
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [4]:
df = df.dropna()
df = df[df.quality == 7]
# df = df[df.relevance == 7] #  和 script 的相關性，但我們不需要管 script ，所以用不到
df.head()
df['origin_index'] = df.index.values

In [5]:
df = df.reset_index(drop=True)
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length,origin_index
0,VPIYF,HJJ4,Recreation room / Man cave,7.0,7.0,Yes,A person is standing by a shelf and grasping h...,phone;shelf,A person takes a phone off of a shelf and look...,c015 1.80 11.00,10.29,8
1,INQNU,H8N1,Bedroom,7.0,7.0,Yes,A laughing person is fixing a bed in their bed...,bed;blanket;clothes;cup;dish;glass;water,"A person is tidying up a bed, they then sit do...",c107 10.20 17.30;c135 8.60 14.60;c151 8.80 14....,18.46,11
2,LI6LV,Z68L,Bathroom,7.0,7.0,Yes,"In the bathroom, the person put a towel on the...",clothes;glass;jacket;medicine;mirror;sink;swea...,A person holding a towel places it on a hook. ...,c034 0.00 6.70;c129 30.20 36.00;c106 31.40 36....,34.71,12
3,DOQ9Y,EA2K,Pantry,7.0,7.0,Yes,A person undressing by removing their jacket i...,bag;clothes;door;food;groceries;jacket;shelf,"Person taking off jacket and hanging it up, an...",c001 7.40 15.10;c006 34.00 40.90;c061 17.60 32...,41.29,16
4,4021N,ADN3,Laundry room,7.0,6.0,Yes,A person is tidying the laundry room. They thr...,box;clothes;pillow;trash can,A person walks into a laundry room carrying a ...,c040 8.00 27.80;c042 24.00 28.70;c076 0.00 9.9...,30.33,26


In [6]:
dat_id = df['id']
dat_actions = df['actions']
dat_origin_index = df['origin_index']
dat = pd.concat([dat_id, dat_origin_index, dat_actions], axis=1)
split_arr = dat['actions'].str.split(';')

In [7]:
large_arr = []
for element in enumerate(split_arr):
    arr = []
    for i in element[1]:
        k = i.split(" ")
        d = k[0]
        arr.append(d)
    large_arr.append(arr)
dat['split'] = large_arr

In [12]:
dat = dat.drop('actions', 1)

In [13]:
dat.head()

,id,origin_index,split
0,VPIYF,8,[c015]
1,INQNU,11,"[c107, c135, c151, c106, c152, c004, c075, c118]"
2,LI6LV,12,"[c034, c129, c106, c155, c148, c129, c001, c09..."
3,DOQ9Y,16,"[c001, c006, c061, c097, c155, c081, c141, c02..."
4,4021N,26,"[c040, c042, c076, c039, c041, c043, c000]"


In [29]:
str_actions = dat['split']
int_actions = []
for action in str_actions:
    arr = []
    for act in action:
        a = int(act[1:])
        arr.append(a)
    int_actions.append(arr)

In [31]:
str_actions

0                                                  [c015]
1        [c107, c135, c151, c106, c152, c004, c075, c118]
2       [c034, c129, c106, c155, c148, c129, c001, c09...
3       [c001, c006, c061, c097, c155, c081, c141, c02...
4              [c040, c042, c076, c039, c041, c043, c000]
5                          [c044, c000, c154, c033, c035]
6                                [c098, c155, c100, c003]
7       [c055, c061, c063, c156, c132, c059, c119, c12...
8                                [c015, c092, c087, c097]
9       [c009, c026, c006, c008, c097, c028, c030, c15...
10      [c132, c072, c155, c151, c070, c061, c073, c15...
11      [c062, c063, c059, c011, c009, c020, c018, c01...
12      [c026, c027, c032, c051, c059, c116, c115, c11...
13                         [c061, c063, c113, c139, c114]
14       [c152, c151, c123, c009, c062, c014, c052, c011]
15                               [c040, c043, c096, c150]
16                                     [c149, c153, c098]
17            